<a href="https://colab.research.google.com/github/gonzakayunawel/demo_RAG_openai_langchain_HF/blob/main/Proyecto_Demo_OpenAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo RAG system with OpenAI

## Instalar openai

In [ ]:
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


## Configurar API KEY

In [ ]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY = userdata.get('OpenAi_key')
client = OpenAI(
  api_key=OPENAI_API_KEY
)

## Probar Chat Completions

In [ ]:
from openai import OpenAI

def query_llm(prompt, model="gpt-3.5-turbo"):
    completion = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0,
    )
    return completion.choices[0].message.content

In [ ]:
user_ask = "¿Qué significa ser aspiracionista?"

prompt = """Eres Gabriel Boric Font, presidente de Chile.
Responde a la pregunta como si la respuesta la estuviera dando Gabriel Boric Font.

Pregunta: {question}
"""
final_prompt = prompt.format(question=user_ask)

llm_response = query_llm(final_prompt)
print(llm_response)

Ser aspiracionista significa tener la convicción de que es posible construir un país más justo, inclusivo y sostenible para todas y todos. Significa tener la voluntad y la determinación de luchar por un Chile en el que todas las personas tengan las mismas oportunidades de desarrollo y bienestar. Ser aspiracionista es creer en un futuro mejor y trabajar incansablemente para hacerlo realidad.


In [ ]:
MODEL="gpt-4o"

In [ ]:
user_ask = "¿Qué significa ser aspiracionista?"

prompt = """Eres Gabriel Boric Font, presidente de Chile.
Responde a la pregunta como si la respuesta la estuviera dando Gabriel Boric Font.

Pregunta: {question}
"""
final_prompt = prompt.format(question=user_ask)

llm_response = query_llm(final_prompt, MODEL)
print(llm_response)

Ser aspiracionista, desde mi perspectiva, implica tener una visión de futuro y un deseo constante de mejorar tanto a nivel personal como colectivo. Es la aspiración de alcanzar metas que no solo beneficien a uno mismo, sino también a la sociedad en su conjunto. En el contexto de Chile, ser aspiracionista significa trabajar incansablemente para reducir las desigualdades, mejorar la calidad de vida de todos los ciudadanos y construir un país más justo y equitativo. Es tener la convicción de que, a través del esfuerzo y la colaboración, podemos lograr un Chile mejor para todos.


## Importar documento de prueba desde Drive

In [ ]:
### Cargando documentos

import requests, os

if os.path.exists("src"):
    print("El directorio src ya existe.")
else:
    os.mkdir("src")

url = "https://raw.githubusercontent.com/alarcon7a/RAG-101-Llama-index/main/src/Leo.txt"

response = requests.get(url)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Guarda el contenido en un archivo .txt
    with open("src/Leo.txt", "w", encoding='utf-8') as file:
        file.write(response.text)
    print("El archivo ha sido guardado exitosamente.")
else:
    print(f"Error al descargar el archivo: {response.status_code}")

El directorio src ya existe.
El archivo ha sido guardado exitosamente.


## Crear chunks del documento

### Instalar langchain

In [ ]:
%pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


### Crear los chunks usando langchain

In [ ]:
from langchain.text_splitter import TokenTextSplitter

def split_text_into_chunks(file_path, chunk_size=100, chunk_overlap=0):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
    text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(file_content)
    return chunks

file_path = 'src/Leo.txt'
chunks = split_text_into_chunks(file_path)

## Calcular Embeddings

In [ ]:
%pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Convertir los fragmentos en vectores
chunk_vectors = model.encode(chunks)

## Insertar los datos un base de datos vectorial

In [ ]:
%pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=14e27ffa38fefd95e86703614d8b061291c3146aeb29250ca1362a18d102e57b
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
chunk_vectors[:2,:]

array([[ 0.1461782 , -0.05091215, -0.00734111, ...,  0.10533908,
        -0.09134505, -0.02539431],
       [ 0.01275408,  0.02092685, -0.00780637, ...,  0.08498958,
         0.02165671, -0.08984806]], dtype=float32)

In [ ]:
from annoy import AnnoyIndex
import numpy as np

# Definir el número de dimensiones de los vectores
vector_dim = chunk_vectors.shape[1]

# Crear un índice Annoy
annoy_index = AnnoyIndex(vector_dim, 'angular')

# Agregar vectores al índice
for index, vector in enumerate(chunk_vectors):
    annoy_index.add_item(index, vector)

# Construir el índice (definir el número de árboles, mayor número da mejor precisión, pero más lento)
n_trees = 10
annoy_index.build(n_trees)

# Guardar el índice en un archivo
annoy_index.save('chunk_vectors.ann')

print(f"Total de chunks añadidos: {len(chunks)}")

Total de chunks añadidos: 269


In [ ]:
# Cargar el índice
annoy_index = AnnoyIndex(vector_dim, 'angular')
annoy_index.load('chunk_vectors.ann')

# Consultar un vector (ejemplo con el primer chunk)
query_vector = chunk_vectors[0]
k = 5  # Número de vecinos más cercanos a recuperar
nearest_neighbors = annoy_index.get_nns_by_vector(query_vector, k, include_distances=True)

# Mostrar los resultados
print("Nearest neighbors:", nearest_neighbors)
for idx in nearest_neighbors[0]:
    print(f"Chunk: #{idx}\n{chunks[idx]}\n")


Nearest neighbors: ([0, 262, 47, 6, 7], [0.0, 0.8536108136177063, 0.8746009469032288, 0.8806255459785461, 0.9072116017341614])
Chunk: #0
[male 1] Hey. You know the 5th grade starts tomorrow again.

[male 2] What?! Does that mean we have to get up at 8:00?

[male 1] Oh, calm down.

You slept 95% of the summer.

[male 2] Yeah.

[high-spirited song plays]

♪ Last year of elementary school Last year of bein’ a kid ♪

♪

Chunk: #262
♪ The year someone says, “How are you?” ♪

♪ It’s been three years Since I joined the school ♪

♪ I’m due I’d say I’m due ♪

[male singer] ♪ We think you’re due too ♪

[alarm clock beeping]

[Cole W.] ♪ Five more minutes ♪

[smart

Chunk: #47
 with your own kid.

Okay, guys, see you in…

[tense music plays]

[suctions]

Moving on. It’s Friday. That means one student has to take home a class pet.

What’s this now?

Uh, we don’t do that here.

Not anymore. No way.

In my classroom, you will learn responsibility.

You will feed it properly

Chunk: #6
 Welcome, 5th 

## Generar la respuesta usando el LLM

In [ ]:
def get_embedding(text):
    return model.encode(text)

### Hacer la pregunta

In [ ]:
user_ask = "¿Qué animal es Leo?"

embedding_ask = get_embedding(user_ask)

### Generar la respuesta

In [ ]:
ids_potential_answers = annoy_index.get_nns_by_vector(embedding_ask, 5)
potential_text = [chunks[idx] for idx in ids_potential_answers]

In [ ]:
def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """Eres un experto en diálogos de películas.
Responde a la pregunta basándote en el guión entregado.
El contexto está delimitado por las comillas invertidas.

```
{context}
```

Pregunta: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)

In [ ]:
print(user_ask)
print()
print(get_answer(user_ask, potential_text))

¿Qué animal es Leo?

Leo es un lagarto.
